<a href="https://colab.research.google.com/github/zrghassabi/DeepLearning/blob/main/Fine_tuning%5B1%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning - Fine-tuning



## Fine Tuning

In [1]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
    ])

trainset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=True, download=True,transform=transform)
testset = datasets.CIFAR10(root='~/.pytorch/CIFAR10',train=False, transform=transform)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:19<00:00, 8707587.60it/s]


Extracting /root/.pytorch/CIFAR10/cifar-10-python.tar.gz to /root/.pytorch/CIFAR10


In [6]:
for images, labels in trainloader:
  print(images.size(), labels.size())
  break

torch.Size([64, 3, 224, 224]) torch.Size([64])


In [7]:
model = models.vgg16(pretrained=True)
model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 173MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
for param in model.parameters():
  param.requires_grad = False

In [9]:
for i in range(0,7):
  model.classifier[i].requires_grad = True

In [10]:
model.classifier[6] = nn.Sequential(
                      nn.Linear(4096,512),
                      nn.ReLU(),
                      nn.Dropout(0.5),
                      nn.Linear(512,10),
                      nn.LogSoftmax(dim=1)
                      )


In [11]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [12]:
criterion = nn.NLLLoss()

In [13]:
from torch.optim import Adam

model = model.to(device)
optimizer = Adam(model.parameters())


## Training from the Fully Connected Network onwards

### Re-training the model

In [14]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0

  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')

  print(f'Training loss : {batch_loss/len(trainloader)}')


Epoch(0/1 : Batch number(1/782) : Batch loss : 2.3620846271514893
Epoch(0/1 : Batch number(2/782) : Batch loss : 2.1389474868774414
Epoch(0/1 : Batch number(3/782) : Batch loss : 2.0060489177703857
Epoch(0/1 : Batch number(4/782) : Batch loss : 1.6810866594314575
Epoch(0/1 : Batch number(5/782) : Batch loss : 1.5945062637329102
Epoch(0/1 : Batch number(6/782) : Batch loss : 1.3893449306488037
Epoch(0/1 : Batch number(7/782) : Batch loss : 1.246921420097351
Epoch(0/1 : Batch number(8/782) : Batch loss : 1.207769513130188
Epoch(0/1 : Batch number(9/782) : Batch loss : 1.0705920457839966
Epoch(0/1 : Batch number(10/782) : Batch loss : 1.3266772031784058
Epoch(0/1 : Batch number(11/782) : Batch loss : 1.0323084592819214
Epoch(0/1 : Batch number(12/782) : Batch loss : 1.1073334217071533
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.9151110649108887
Epoch(0/1 : Batch number(14/782) : Batch loss : 1.0256471633911133
Epoch(0/1 : Batch number(15/782) : Batch loss : 1.0783385038375854
Epoch(

### The accuracy of the model

In [16]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):

        logps = model(images)
        output = torch.exp(logps)

        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')

        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 78.75% 


## Un-freezing & training on the LAST CNN block onwards

In [17]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

### Re-training the model

In [18]:
for i in range(24,31):
  model.features[i].requires_grad = True


In [19]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 3
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0

  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')

  print(f'Training loss : {batch_loss/len(trainloader)}')


Epoch(0/3 : Batch number(1/782) : Batch loss : 0.46183860301971436
Epoch(0/3 : Batch number(2/782) : Batch loss : 0.4039269685745239
Epoch(0/3 : Batch number(3/782) : Batch loss : 0.34544309973716736
Epoch(0/3 : Batch number(4/782) : Batch loss : 0.5524992346763611
Epoch(0/3 : Batch number(5/782) : Batch loss : 0.4260405898094177
Epoch(0/3 : Batch number(6/782) : Batch loss : 0.5770209431648254
Epoch(0/3 : Batch number(7/782) : Batch loss : 0.5803196430206299
Epoch(0/3 : Batch number(8/782) : Batch loss : 0.3721299171447754
Epoch(0/3 : Batch number(9/782) : Batch loss : 0.40232014656066895
Epoch(0/3 : Batch number(10/782) : Batch loss : 0.5816544890403748
Epoch(0/3 : Batch number(11/782) : Batch loss : 0.4967535734176636
Epoch(0/3 : Batch number(12/782) : Batch loss : 0.4799668788909912
Epoch(0/3 : Batch number(13/782) : Batch loss : 0.6389386653900146
Epoch(0/3 : Batch number(14/782) : Batch loss : 0.3240217864513397
Epoch(0/3 : Batch number(15/782) : Batch loss : 0.438296914100647
Ep

### The accuracy of the model

In [20]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):

        logps = model(images)
        output = torch.exp(logps)

        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')

        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
Batch (5/157)
Accuracy of the model on 320 test images: 82.8125% 


## Un-freezing & training on the LAST TWO CNN block onwards

### Re-training the model

In [21]:
for i in range(17,24):
  model.features[i].requires_grad = True

In [22]:
model = model.to(device)
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()))

num_epochs = 1
batch_loss = 0
cum_epoch_loss = 0

for e in range(num_epochs):
  cum_epoch_loss = 0

  for batch, (images, labels) in enumerate(trainloader,1):
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    batch_loss += loss.item()
    print(f'Epoch({e}/{num_epochs} : Batch number({batch}/{len(trainloader)}) : Batch loss : {loss.item()}')

  print(f'Training loss : {batch_loss/len(trainloader)}')


Epoch(0/1 : Batch number(1/782) : Batch loss : 0.255924254655838
Epoch(0/1 : Batch number(2/782) : Batch loss : 0.31262004375457764
Epoch(0/1 : Batch number(3/782) : Batch loss : 0.14360855519771576
Epoch(0/1 : Batch number(4/782) : Batch loss : 0.19025444984436035
Epoch(0/1 : Batch number(5/782) : Batch loss : 0.29372578859329224
Epoch(0/1 : Batch number(6/782) : Batch loss : 0.2155156135559082
Epoch(0/1 : Batch number(7/782) : Batch loss : 0.22661375999450684
Epoch(0/1 : Batch number(8/782) : Batch loss : 0.2384774535894394
Epoch(0/1 : Batch number(9/782) : Batch loss : 0.13092872500419617
Epoch(0/1 : Batch number(10/782) : Batch loss : 0.23235446214675903
Epoch(0/1 : Batch number(11/782) : Batch loss : 0.1372067779302597
Epoch(0/1 : Batch number(12/782) : Batch loss : 0.1922730952501297
Epoch(0/1 : Batch number(13/782) : Batch loss : 0.148966982960701
Epoch(0/1 : Batch number(14/782) : Batch loss : 0.21818219125270844
Epoch(0/1 : Batch number(15/782) : Batch loss : 0.239128842949867

### The accuracy of the model

In [ ]:
model.to('cpu')

model.eval()
with torch.no_grad():
    num_correct = 0
    total = 0

    #set_trace()
    for batch, (images, labels) in enumerate(testloader,1):

        logps = model(images)
        output = torch.exp(logps)

        pred = torch.argmax(output, 1)
        total += labels.size(0)
        num_correct += (pred == labels).sum().item()
        print(f'Batch ({batch}/{len(testloader)})')

        if batch == 5:
          break

    print(f'Accuracy of the model on {total} test images: {num_correct * 100 / total}% ')

Batch (1/157)
Batch (2/157)
Batch (3/157)
Batch (4/157)
